In [1]:
import pandas as pd 
import gensim

In [38]:
#gensim is an NLP library

In [36]:
#word2Vec has two main approaches 
#1. CBOW
#2. Skip-gram

In [3]:
df = pd.read_json(r"C:\Users\Ankit\reviews_Cell_Phones_and_Accessories_5.json",lines=True)

In [5]:
df.head(7)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
5,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daug...,3,Cute,1381536000,"10 12, 2013"
6,A1JVVYYO7G56DS,120401325X,Tyler Evans,"[0, 0]",Came just as described.. It doesn't come unstu...,5,best thing ever..,1377129600,"08 22, 2013"


In [8]:
#a very big file
df.shape

(194439, 9)

In [14]:
#as you can see this sentence has unncessary stuff like "",!. , first we need to remove this using gensim and also 
#at the saem time , gensim will makke the capital letters to lower to be more convienient !
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [17]:
#gensim.utils.simple_preprocess , this method will automatically does this 

review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [16]:
#now you can seee that gensim removes the unnecessary entities!
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [37]:
#as we know that to do cbow or skipgram  we have this sliding window which has context and target 
# in this case taking 10 , means 10 words before the target word and after the target word
#min_count =2 means , the sentence should at least have 2 words , it will not going to consider a single word sentence
#workers = 4 means no. of CPU 's you have that to wanted to run 
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [40]:
#building Vocabulary :- means it will find out unique elts. 
#for eg:_  If a word appears 1000 times, it is still stored only once in memory — with a single vector.

In [39]:
model.build_vocab(review_text, progress_per=1000)

In [27]:
#bydefault they are running 5 epochs
model.epochs

5

In [26]:
#corpus_count have the total no. of samples
model.corpus_count

194439

In [22]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61502194, 83868975)

In [23]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [30]:
#wv.most_similar will find the words have the same feactuee vectors !!
model.wv.most_similar("bad")

[('terrible', 0.6628029346466064),
 ('horrible', 0.602308452129364),
 ('shabby', 0.6008577942848206),
 ('good', 0.5764705538749695),
 ('lame', 0.5415282249450684),
 ('cheap', 0.5314403772354126),
 ('mad', 0.5276988744735718),
 ('disappointing', 0.5247689485549927),
 ('awful', 0.5156147480010986),
 ('okay', 0.5153468251228333)]

In [32]:
#through wv.similarity we find how much similar is this two words as per their feature vectors !
model.wv.similarity(w1="cheap", w2="inexpensive")

0.5309235

In [34]:
#as they both are same word , therefore the similarity score is of 1 
model.wv.similarity(w1="cheap", w2="cheap")

1.0